# Home assignment 4

##### by Anastasiia Khaburska

Побудуйте класифікатор, який буде предбачати категорію запиту до служби 1551 з використанням тільки векторів слів (не використовувати інших фічі, такі як самі слова, нграми і т.д).


Вектори можна завантажити зі [сторінки проекту lang-uk](http://lang.org.ua/en/models/#anchor4) - можете обрати будь-які з векторів.


Запити знаходяться у архіві [1551.zip](https://github.com/vseloved/prj-nlp-2019/blob/master/tasks/1551.zip). Кожний файл в архіві має назву, яка відповідає категорії запитів, яку треба передбачати (загалом, 188 категорій), та містить запити у формі: id, текст запиту, 2 порожніх рядки.

Можете використайте фільтрацію мови, щоб відібрати тільки україномовні запити.


1. У якості безйлайну використайте класифікатор kNN, який буде шукати найближчі за косинусною відстанню вектори запитів, утворені сумуванням векторів всіх слів запиту.


2. Розробіть покращений класифікатор, для якого використайте іншу модель класифікації, яка би присвоювала ваги окремим елементам вектора (наприклад, Logistic Regression, SVM, Random Forest). Також можете зробити додаткові покращення, такі як:

    - здійснювати попередню фільтрацію і відбирати тільки частину векторів для представлення

    - використовувати лематизацію та інші форми нормалізації

    - робити зваження векторів

    - додатково використовувати інші способи побудувати вектор документу (наприклад, TF-IDF, LDA, ...)


3. Розробіть класифікатор на основі нейромережі: або feed-forward з використанням агрегованого вектору, або LSTM (чи BiLSTM), що проходить по всьому тексту. 


Порівняйте між собою якість роботи класифікаторів. Для навчання створіть вибірку, що міститиме 70% текстів, а для поріняння використовуйте залишок 30%.


Оцінювання: 30 балів + 30 балів + 40 балів

### Короткий опист зробленого:

(на кожному кроці робила pickle отриманого результату, для того, щобю швидше переранювати документ. Оригінал коду *$# коменти$* достатньо забрати знак $#$ щоб виконати все з нуля)

Усі обчислення проводила на вибірці 500_3000 (якщо клас представлений більшою кількістю ніж 500 повідомлень і меншою ніж 3000. Таких класів є 22 з 188), оскільки обчислювальні можливості мого компютера дуже обмежені і я вирішила використати більше часу на дослідження та випробовування великої кількості покращень та моделей. Я свідома того, що якщо проводити обчислення на всій вибірці, то результати сильно відрізнятимуться, оскільки датасет буде набагато менш збалансованим. Я припускаю, що на повному (і незбалансованому) датасеті найкращий результат показав би ансамбль різних методів, а не чистий SVM з TF-IDF extracted features, як у мене.
        
0. Препроцесінг даних:
    - Зчитала всі повідомлення з файлів та записала їх у DataFrame (**Read data**)
    - Завантажила векторне представдення слів української мови *LexVec* та *Word2Vec* (**LexVec vs Word2Vec**)
    - Обчислила ембедінг слів у повідомленнях, також зробивши мінімальну чистку даних(видалила стоп слова, дефізи, видалила повідомлення без векторів, які є в словнику, видалила дублікати. Обчислила для кожного документу два варіанти векторів 1)середній 2)суму векторів. Як показали експеременти, сума векторів дає кращі результати (**Doc2Vectors**) 
    - Розділила на train i test з допомогою sklearn.model_selection.train_test_split(**Train/Test split**)
    
    
1. Обчислила baseline KNN model:
    - KNN_cosine_500_3000 (metric=cosine, вектор середній) | accuracy: 45%
    - KNN_S_500_3000 (metric=mincowski, вектор сума) | accuracy: 60%
    - KNN_S_cosine_500_3000_10 (=cosine, вектор сума) | accuracy: 67.05%
    - =>  як baseline обрала KNN_S_cosine_500_3000_10


2. Як покращений клаисфікатор спробувала застосувати:
    - Logistic Regression model (вектор сума)              | accuracy: 72.41%
    - LR + TF-IDF feature extraction                       | accuracy: 76.57%
    - Random Forest Classifier + TF-IDF                    | accuracy: 69%
    - Random Forest Classifier + LexVec (вектор сума)      | accuracy: 60%
    - SVM model (TF-IDF features)                          | accuracy: 82.31 %
    - SVM model (doc2vecS features) вектор сума)           | accuracy: 71.39 %
   
   
3. Класифікатори на основі нейронних мереж:
    - LSTM model + LexVec (вектор сума) | accuracy: 68.89 %
    - Feedforward NN model + LexVec (вектор сума) | accuracy: 74.06 %
    
    
4. !!! Ensemble model ('KNN','LR_TFIDF', 'RF_TFIDF','SVM_TFIDF','NN_Doc2VecS')+ LexVec (вектор сума) | accuracy: 80.85 %
    - *SVM_TFID* has to votings
    - думаю можна покращити видаливши KNN i одавши LSTM до ансамблю, але, оскільки ансамбль обчислюється дуже довго і $y$ в LSTM є dummies, лишу це на TODO, якщо колись матиму час
    

In [1]:
import os
import re
import pickle
import tokenize_uk
import glob
import pymorphy2
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.neighbors import KNeighborsClassifier
pd.set_option('display.max_colwidth', 300)
import numpy as np
from nltk import word_tokenize
from collections import Counter
from langdetect import detect
from scipy.spatial.distance import cosine
from sklearn.model_selection import train_test_split
from langdetect.lang_detect_exception import LangDetectException
from tqdm import tqdm
from sklearn.metrics import classification_report

### Read data

In [2]:
def get_files(directory_path):
    '''Read all letters in the given directory to a dataframe
    category, text, id
    only ukrainian letters'''
    
    data = []
    for filename in tqdm(os.listdir(directory_path)):
        with open(os.path.join(directory_path, filename)) as f:
            text=f.read()
            #first id in a file
            id_first_in_file= [re.search(r'\d{6,8}', text).group()]
            #delete first id
            text=re.sub(r'\d{6,8}\n', "", text, 1)
            #split letters by \n\nid pattern and make a list of letters
            letters = re.split(r'\n{2,4}\d{6,8}\n', text)
            #make a list of ids
            idies = id_first_in_file+re.findall(r'\n{2,4}\d{6,8}\n', text)
            #get category from the filename
            category = filename[:len(filename)-4]
            #append to the main list
            for (letter, ide) in zip(letters, idies):
                if letter.strip('\n') and (detect(letter) == 'uk'):
                    data.append((category, re.sub(r'\n', " ", letter), ide.strip('\n')))
    return pd.DataFrame(data, columns=['category', 'text','id'])

#df_data = get_files('1551')
#df_data.to_csv ("letters.csv", index = None)
#df_data.head(10)

In [3]:
letters=pd.read_csv('letters.csv')
letters.head(5)

,category,text,id
0,Ремонт-теплотрас--підключення,"24 травня 2015 року ( неділю) вранці- у подвір’ і між будинками 42 і 44 вул . Артема- ескаватор під наглядом працівників Киівенерго( що підтверджує наявність машини Київенерго)- розрили величезну яму, чим самим заблокували проїзд автотранспорту у подвір’я. Ніякі роботи протягом дня (24 травня) н...",3171804
1,Ремонт-теплотрас--підключення,"Доброго дня вельмишановні працівники Контактцентру та державні службовці. Всі мої звернення, які зареєстровані, стосуються однієї проблеми - тепломережі, прибудинкової території та якість виконання робіт. Вважаючи на результати та наявний стан справ, вони, як і мабуть всі інші, сумлінно не викон...",3059902
2,Ремонт-теплотрас--підключення,Доброго дня! Прошу надати роз’яснення з приводу ремонтних робіт теплотраси перед будинком а саме: 1) Чому ремонтні роботи тривають так довго (більше як 1.5 місяця)? Робітники і техніка виконували роботи не більше 5-7 днів. 2) Коли планове завершення? Яму з кожним разом розривають все більше. 3)...,3005500
3,Ремонт-теплотрас--підключення,"Добрий вечір. Звертались до вас тиждень тому з приводу відсутності гарячої води та отопленя, нам сказали що 21числа має все бути, зараз 22 число, не має ні тепла, ні гарячої води. До ЖКГ не можливо дозвонитися. Проблема з гарячою водою в нас завжди, виключають після 23.00 або ж взагалі немає. Не...",2618062
4,Ремонт-теплотрас--підключення,"Доброго дня. Хочу офіційно заявити скаргу про незадовільне, неякісне та злочинне, щодо технології виконання робіт, їх якості і використання бюджетних коштів, які були проведені щодо усунення недоліків зазначених мною у зверненні Г-14867 від 25 жовтня 2014 року. Ремонт теплотраси відбувся методо...",2988291


In [4]:
len(letters)

61818

### LexVec vs Word2Vec

In [6]:
import logging
logger = logging.getLogger("imported_module")

In [7]:
logger.setLevel(logging.CRITICAL)

uk_w2vectors_file = 'news.lowercased.tokenized.word2vec.300d'
uk_vectors_file = 'news.lowercased.tokenized.300d'

uk_w2vectors = KeyedVectors.load_word2vec_format(uk_w2vectors_file, binary=False)
uk_vectors = KeyedVectors.load_word2vec_format(uk_vectors_file, binary=False)

In [8]:
king_vectors = uk_vectors.get_vector('король')
man_vectors = uk_vectors.get_vector('чоловік')
woman_vectors = uk_vectors.get_vector('жінка')

print(uk_vectors.most_similar(positive=['жінка', 'король'], negative=['чоловік']))
print()
print(uk_vectors.similar_by_vector(king_vectors - man_vectors + woman_vectors))

/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:2377: RuntimeWarning: invalid value encountered in true_divide
  return (m / dist).astype(REAL)


[('королева', 0.6227983236312866), ('принцеса', 0.552274763584137), ('короля', 0.4998285174369812), ('красуня', 0.4710363745689392), ('королем', 0.4370134472846985), ('зірка', 0.4352923333644867), ('принц', 0.43263137340545654), ('дівчина', 0.41159868240356445), ('монархія', 0.40422725677490234), ('королеви', 0.39589250087738037)]

[('король', 0.8295454978942871), ('королева', 0.63805091381073), ('принцеса', 0.5543002486228943), ('короля', 0.5509599447250366), ('принц', 0.49959513545036316), ('королем', 0.4934990108013153), ('красуня', 0.46002262830734253), ('зірка', 0.4264647960662842), ('королеви', 0.4191911220550537), ('монарх', 0.4119150638580322)]


In [9]:
king_w2vectors = uk_w2vectors.get_vector('король')
man_w2vectors = uk_w2vectors.get_vector('чоловік')
woman_w2vectors = uk_w2vectors.get_vector('жінка')

print(uk_w2vectors.most_similar(positive=['жінка', 'король'], negative=['чоловік']))
print()
print(uk_w2vectors.similar_by_vector(king_w2vectors - man_w2vectors + woman_w2vectors))

[('королева', 0.6513357162475586), ('принцеса', 0.5810298323631287), ('дама', 0.5590238571166992), ('танцівниця', 0.5500212907791138), ('кінозірка', 0.5444834232330322), ('знаменитість', 0.5426367521286011), ('матуся', 0.5346686840057373), ('красуня', 0.5342152714729309), ('дівчина', 0.5334903597831726), ('імператриця', 0.5316278338432312)]

[('королева', 0.6083585023880005), ('дама', 0.5521167516708374), ('принцеса', 0.5491373538970947), ('знаменитість', 0.544516921043396), ('жінка', 0.5441809892654419), ('танцівниця', 0.5311182737350464), ('панянка', 0.5286626815795898), ('кінозірка', 0.5274341106414795), ('актриса', 0.52556973695755), ('матуся', 0.5253670811653137)]


### Doc2Vectors

In [10]:
def read_stopwords_from_file(filename):
    with open(filename) as stopfile:
        stopwords = stopfile.read()
        stop_list = stopwords.split()
    return stop_list

In [11]:
#1
#from stop_words import get_stop_words
#stop_words = get_stop_words('ukrainian')
#2
#stop_words = stopwords.words('uk')
#3
stop_words=read_stopwords_from_file("stopwords_ukrainian")
print(stop_words)

['а', 'або', 'але', 'б', 'без', 'би', 'бо', 'був', 'буде', 'була', 'були', 'було', 'бути', 'в', 'вам', 'вами', 'вас', 'ваш', 'ваша', 'ваше', 'вашим', 'вашими', 'ваших', 'ваші', 'вашій', 'вашого', 'вашої', 'вашому', 'вашою', 'вашу', 'вже', 'ви', 'від', 'він', 'вона', 'вони', 'воно', 'всі', 'де', 'для', 'до', 'дуже', 'є', 'з', 'за', 'зі', 'і', 'із', 'її', 'їй', 'їм', 'їх', 'й', 'його', 'йому', 'ким', 'кого', 'коли', 'кому', 'лише', 'має', 'мене', 'мені', 'ми', 'мій', 'мною', 'мого', 'моє', 'моєї', 'моєму', 'моєю', 'можна', 'мої', 'моїй', 'моїм', 'моїми', 'моїх', 'мою', 'моя', 'на', 'нам', 'нами', 'нас', 'наш', 'наша', 'наше', 'нашим', 'нашими', 'наших', 'наші', 'нашій', 'нашого', 'нашої', 'нашому', 'нашою', 'нашу', 'неї', 'нею', 'ним', 'ними', 'них', 'ній', 'нім', 'ну', 'нього', 'ньому', 'під', 'після', 'по', 'при', 'про', 'саме', 'себе', 'собі', 'та', 'так', 'також', 'там', 'твій', 'твого', 'твоє', 'твоєї', 'твоєму', 'твоєю', 'твої', 'твоїй', 'твоїм', 'твоїми', 'твоїх', 'твою', 'твоя', 

In [12]:
morph = pymorphy2.MorphAnalyzer(lang='uk')
def doc2vec(text, vectors):
    text=re.sub(r'[.\-\d]', " ", text)
    tokens = [morph.parse(i.lower().strip(r"\d"))[0].normal_form for i in word_tokenize(text)
                  if morph.parse(i.lower()) and not i.lower().strip("-") in stop_words]
    tokens_vectors = np.array([vectors.get_vector(i)  for i in tokens if i in vectors.vocab])
    if tokens_vectors.size!=0:
        max_vect = tokens_vectors.max(axis=0)
        avg_vect =  tokens_vectors.sum(axis=0) / tokens_vectors.sum(axis=0).sum()
        doc_vect = np.concatenate((max_vect,avg_vect))
        return doc_vect.reshape(1, -1)
    else:
        return 'Empty'
    
def doc2vecS(text, vectors):
    text=re.sub(r'[.\-\d]', " ", text)
    tokens = [morph.parse(i.lower().strip(r"\d"))[0].normal_form for i in word_tokenize(text)
                  if morph.parse(i.lower()) and not i.lower().strip("-") in stop_words]
    tokens_vectors = np.array([vectors.get_vector(i)  for i in tokens if i in vectors.vocab])
    if tokens_vectors.size!=0:
        sum_vect =  tokens_vectors.sum(axis=0)
        doc_vect = sum_vect
        return doc_vect.reshape(1, -1)
    else:
        return 'Empty'

In [13]:
def similar_doc(doc, docs, vec_docs, w2=1):
    vec_sim = np.array([cosine_similarity(doc2vec(doc)[w2], vec_doc) for vec_doc in vec_docs])
    return docs[vec_sim.argmax()]

def similartovec_doc(vecdoc, docs, vec_docs, w2=1):
    vec_sim = np.array([cosine_similarity(vecdoc[w2], vec_doc) for vec_doc in vec_docs])
    return docs[vec_sim.argmax()]

def similar_docS(doc, docs, vec_docs):
    vec_sim = np.array([cosine_similarity(doc2vecS(doc), vec_doc) for vec_doc in vec_docs])
    return docs[vec_sim.argmax()]

In [14]:
#df_data=pd.read_csv('letters.csv')
#df_data.drop_duplicates(subset ="text", keep = False, inplace = True)
#df=df_data.copy()
#df.loc[:, 'doc2vec'] = df.apply(lambda x: doc2vec(x['text'], uk_vectors), axis=1)
#df.loc[:, 'doc2w2vec'] = df.apply(lambda x: doc2vec(x['text'], uk_w2vectors), axis=1)
#df = df[df['doc2vec'] != 'Empty']
#df = df[df['doc2w2vec'] != 'Empty']
#df = df[df['doc2vec'].map(lambda x: False if np.any(np.isnan(x)) else True)]
#df = df[df['doc2w2vec'].map(lambda x: False if np.any(np.isnan(x)) else True)]
#df.loc[:, 'doc2vec_reshape'] = df['doc2vec'].map(lambda x: x[0])
#df.loc[:, 'doc2w2vec_reshape'] = df['doc2w2vec'].map(lambda x: x[0])

#df.loc[:, 'doc2vecS'] = df.apply(lambda x: doc2vecS(x['text'], uk_vectors), axis=1)
#df.loc[:, 'doc2w2vecS'] = df.apply(lambda x: doc2vecS(x['text'], uk_w2vectors), axis=1)
#df = df[df['doc2vecS'] != 'Empty']
#df = df[df['doc2w2vecS'] != 'Empty']
#df = df[df['doc2vecS'].map(lambda x: False if np.any(np.isnan(x)) else True)]
#df = df[df['doc2w2vecS'].map(lambda x: False if np.any(np.isnan(x)) else True)]
#df.loc[:, 'doc2vecS_reshape'] = df['doc2vecS'].map(lambda x: x[0])
#df.loc[:, 'doc2w2vecS_reshape'] = df['doc2w2vecS'].map(lambda x: x[0])

*Надалі усі обчислення проводила на вибірці 500_3000 (якщо клас представлений більшою кількістю ніж 500 повідомлень і меншою ніж 3000. Таких класів є 22 з 188), оскільки обчислювальні можливості мого компютера дуже обмежені і я вирішила використати більше часу на дослідження та випробовування великої кількості покращень та моделей. Я свідома того, що якщо проводити обчислення на всій вибірці, то результати сильно відрізнятимуться, оскільки датасет буде набагато менш збалансованим. Я припускаю, що на повному (і незбалансованому) датасеті найкращий результат показав би ансамбль різних методів, а не чистий SVM з TF-IDF extracted features, як у мене :)*

In [15]:
#df.to_pickle("./embedded.pkl")
#unpickled_df = pd.read_pickle("./embedded.pkl")
#df=unpickled_df.copy()
#df_filter = df.groupby('category').count()
#filter_columns = df_filter[(df_filter['text'] >= 500) & (df_filter['text'] <= 3000)].index.values
#df_500_3000 = unpickled_df[unpickled_df['category'].map(lambda x: True if x in filter_columns else False)]
#df_500_3000.to_pickle("./embedded_500_3000.pkl")   
#df.to_pickle("./embedded_500_3000.pkl")
unpickled_df_500_3000 = pd.read_pickle("./embedded_500_3000.pkl")

unpickled_df_500_3000

,category,text,id,doc2vec,doc2w2vec,doc2vec_reshape,doc2w2vec_reshape,doc2vecS,doc2w2vecS,doc2vecS_reshape,doc2w2vecS_reshape
4816,Прибирання-приміщень,В підїзді №1 не прибирається..,3067803,"[[-0.084547, -0.001392, -0.058494, 0.207588, -0.010897, -0.044266, -0.047631, 0.06342, 0.005104, 0.089637, 0.070812, 0.008774, -0.015949, 0.012546, 0.190972, 0.054239, 0.22103, -0.00713, -0.142183, 0.080955, -0.031203, 0.109135, 0.165411, -0.045446, -0.181592, 0.045766, -0.097201, -0.010174, -0....","[[-2.336758, -0.501181, 0.457877, -1.358576, 0.041353, -2.046993, 0.023815, 1.547083, -1.742819, -2.262023, -1.506638, -6.017622, 0.420171, -2.292722, 2.393311, 0.269985, 1.113025, -0.598293, -1.889485, -0.106864, -1.201012, -1.16625, 2.016952, 0.122736, -0.926234, -0.598504, -1.026642, -4.13740...","[-0.084547, -0.001392, -0.058494, 0.207588, -0.010897, -0.044266, -0.047631, 0.06342, 0.005104, 0.089637, 0.070812, 0.008774, -0.015949, 0.012546, 0.190972, 0.054239, 0.22103, -0.00713, -0.142183, 0.080955, -0.031203, 0.109135, 0.165411, -0.045446, -0.181592, 0.045766, -0.097201, -0.010174, -0.0...","[-2.336758, -0.501181, 0.457877, -1.358576, 0.041353, -2.046993, 0.023815, 1.547083, -1.742819, -2.262023, -1.506638, -6.017622, 0.420171, -2.292722, 2.393311, 0.269985, 1.113025, -0.598293, -1.889485, -0.106864, -1.201012, -1.16625, 2.016952, 0.122736, -0.926234, -0.598504, -1.026642, -4.137404...","[[-0.084547, -0.001392, -0.058494, 0.207588, -0.010897, -0.044266, -0.047631, 0.06342, 0.005104, 0.089637, 0.070812, 0.008774, -0.015949, 0.012546, 0.190972, 0.054239, 0.22103, -0.00713, -0.142183, 0.080955, -0.031203, 0.109135, 0.165411, -0.045446, -0.181592, 0.045766, -0.097201, -0.010174, -0....","[[-2.336758, -0.501181, 0.457877, -1.358576, 0.041353, -2.046993, 0.023815, 1.547083, -1.742819, -2.262023, -1.506638, -6.017622, 0.420171, -2.292722, 2.393311, 0.269985, 1.113025, -0.598293, -1.889485, -0.106864, -1.201012, -1.16625, 2.016952, 0.122736, -0.926234, -0.598504, -1.026642, -4.13740...","[-0.084547, -0.001392, -0.058494, 0.207588, -0.010897, -0.044266, -0.047631, 0.06342, 0.005104, 0.089637, 0.070812, 0.008774, -0.015949, 0.012546, 0.190972, 0.054239, 0.22103, -0.00713, -0.142183, 0.080955, -0.031203, 0.109135, 0.165411, -0.045446, -0.181592, 0.045766, -0.097201, -0.010174, -0.0...","[-2.336758, -0.501181, 0.457877, -1.358576, 0.041353, -2.046993, 0.023815, 1.547083, -1.742819, -2.262023, -1.506638, -6.017622, 0.420171, -2.292722, 2.393311, 0.269985, 1.113025, -0.598293, -1.889485, -0.106864, -1.201012, -1.16625, 2.016952, 0.122736, -0.926234, -0.598504, -1.026642, -4.137404..."
4817,Прибирання-приміщень,"1. Прибирання підїзду робиться вкрай нерегулярно. Іноді протягом 3-х тижнів, або навіть місяця. 2. Косметичний ремонт підїзду на останньому поверсі відсутній.",2937422,"[[0.500563, 0.286424, 0.532075, 0.46481, 0.229421, 0.553665, 0.150944, 0.159035, 0.506101, 0.13541, 0.661964, 0.157294, 0.460409, 0.570167, 1.081728, 0.52474, 0.437801, 0.293852, 0.283891, 0.387058, 0.401101, 0.262813, 0.141715, 0.118816, 0.205297, 0.353372, 0.49583, 0.275393, 0.295218, 0.344787...","[[0.341461, 1.204187, 1.925844, 2.655365, 2.204645, 2.012463, 1.843585, 3.687111, 1.211816, 1.546087, 1.323149, 1.915068, 2.61784, 1.563169, 1.592014, 2.853098, 1.864407, 1.802295, 1.56014, 2.49524, 3.138227, 0.259099, 1.280812, 1.664527, 1.057407, 2.424162, 2.103551, 5.108227, 3.080941, 2.61125...","[0.500563, 0.286424, 0.532075, 0.46481, 0.229421, 0.553665, 0.150944, 0.159035, 0.506101, 0.13541, 0.661964, 0.157294, 0.460409, 0.570167, 1.081728, 0.52474, 0.437801, 0.293852, 0.283891, 0.387058, 0.401101, 0.262813, 0.141715, 0.118816, 0.205297, 0.353372, 0.49583, 0.275393, 0.295218, 0.344787,...","[0.341461, 1.204187, 1.925844, 2.655365, 2.204645, 2.012463, 1.843585, 3.687111, 1.211816, 1.546087, 1.323149, 1.915068, 2.61784, 1.563169, 1.592014, 2.853098, 1.864407, 1.802295, 1.56014, 2.49524, 3.138227, 0.259099, 1.280812, 1.664527, 1.057407, 2.424162, 2.103551, 5.108227, 3.080941, 2.6

### Train/Test split

In [16]:
"""X_doc2vec = unpickled_df['doc2vec_reshape'].tolist()
X_doc2w2vec = unpickled_df['doc2w2vec_reshape'].tolist()
X_doc2vecS = unpickled_df['doc2vec'].tolist()
X_doc2w2vecS = unpickled_df['doc2w2vecS'].tolist()
y = unpickled_df['category'].tolist()

X_doc2vec_train, X_doc2vec_test, y_train, y_test = train_test_split(
    X_doc2vec, y, test_size=0.33, random_state=42)
X_doc2w2vec_train, X_doc2w2vec_test, y_train, y_test = train_test_split(
    X_doc2w2vec, y, test_size=0.33, random_state=42)
X_doc2vecS_train, X_doc2vecS_test, y_train, y_test = train_test_split(
    X_doc2vecS, y, test_size=0.33, random_state=42)
X_doc2w2vecS_train, X_doc2w2vecS_test, y_train, y_test = train_test_split(
    X_doc2w2vecS, y, test_size=0.33, random_state=42)"""


"X_doc2vec = unpickled_df['doc2vec_reshape'].tolist()\nX_doc2w2vec = unpickled_df['doc2w2vec_reshape'].tolist()\nX_doc2vecS = unpickled_df['doc2vec'].tolist()\nX_doc2w2vecS = unpickled_df['doc2w2vecS'].tolist()\ny = unpickled_df['category'].tolist()\n\nX_doc2vec_train, X_doc2vec_test, y_train, y_test = train_test_split(\n    X_doc2vec, y, test_size=0.33, random_state=42)\nX_doc2w2vec_train, X_doc2w2vec_test, y_train, y_test = train_test_split(\n    X_doc2w2vec, y, test_size=0.33, random_state=42)\nX_doc2vecS_train, X_doc2vecS_test, y_train, y_test = train_test_split(\n    X_doc2vecS, y, test_size=0.33, random_state=42)\nX_doc2w2vecS_train, X_doc2w2vecS_test, y_train, y_test = train_test_split(\n    X_doc2w2vecS, y, test_size=0.33, random_state=42)"

In [17]:
X_doc2vec_500_3000 = unpickled_df_500_3000['doc2vec_reshape'].tolist()
X_doc2w2vec_500_3000 = unpickled_df_500_3000['doc2w2vec_reshape'].tolist()
X_doc2vecS_500_3000 = unpickled_df_500_3000['doc2vecS_reshape'].tolist()
X_doc2w2vecS_500_3000 = unpickled_df_500_3000['doc2w2vecS_reshape'].tolist()
y_500_3000 = unpickled_df_500_3000['category'].tolist()

X_doc2vec_train_500_3000, X_doc2vec_test_500_3000, y_train_500_3000, y_test_500_3000 = train_test_split(
    X_doc2vec_500_3000, y_500_3000, test_size=0.33, random_state=42)
X_doc2w2vec_train_500_3000, X_doc2w2vec_test_500_3000, y_train_500_3000, y_test_500_3000 = train_test_split(
    X_doc2w2vec_500_3000, y_500_3000, test_size=0.33, random_state=42)
X_doc2vecS_train_500_3000, X_doc2vecS_test_500_3000, y_train_500_3000, y_test_500_3000 = train_test_split(
    X_doc2vecS_500_3000, y_500_3000, test_size=0.33, random_state=42)
X_doc2w2vecS_train_500_3000, X_doc2w2vecS_test_500_3000, y_train_500_3000, y_test_500_3000 = train_test_split(
    X_doc2w2vecS_500_3000, y_500_3000, test_size=0.33, random_state=42)

### KNN classifier

In [18]:
loaded_KNN_cosine_500_3000 = pickle.load(open('netKNN_cosine_500_3000.pkl', 'rb'))
loaded_y_doc2vec_cosine_pred_500_3000= pickle.load(open('y_doc2vec_pred_cosine_500_3000.pkl', 'rb'))

In [19]:
print(classification_report(y_true=y_test_500_3000, y_pred=loaded_y_doc2vec_cosine_pred_500_3000))

                                                                                  precision    recall  f1-score   support

                                              Інші-технічні-недоліки-стану-ліфту       0.19      0.42      0.27       180
                                  Будівництво-дооблаштування-дитячого-майданчику       0.36      0.64      0.46       180
Відновлення-благоустрою-після-вик--планових-аварійних-робіт-на-об-єктах-благоуст       0.40      0.60      0.48       369
                                                                    Відсутнє-ХВП       0.34      0.41      0.37       214
                                                            Відсутність-опалення       0.47      0.63      0.54       831
                                                  Відсутність-опалення-по-стояку       0.34      0.39      0.36       272
          Відсутність-освітлення-у-під-їзді-за-відсутності-несправності-лампочок       0.44      0.49      0.46       369
                       

In [20]:
#KNN_S_500_3000 = KNeighborsClassifier(n_neighbors=3)
#KNN_S_500_3000.fit(X_doc2vecS_train_500_3000, y_train_500_3000) 
#pickle.dump(KNN_S_500_3000, open('netKNN_S_500_3000.pkl', 'wb'))
loaded_KNN_S_500_3000 = pickle.load(open('netKNN_S_500_3000.pkl', 'rb'))
loaded_KNN_S_500_3000

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [21]:
#y_doc2vecS_pred_500_3000 = KNN_S_500_3000.predict(X_doc2vecS_test_500_3000)
#pickle.dump(y_doc2vecS_pred_500_3000, open('y_doc2vecS_pred_500_3000.pkl', 'wb'))
loaded_y_doc2vecS_pred_500_3000 = pickle.load(open('y_doc2vecS_pred_500_3000.pkl', 'rb'))

In [22]:
#print(classification_report(y_true=y_test_500_3000, y_pred=y_doc2vecS_pred_500_3000))
print(classification_report(y_true=y_test_500_3000, y_pred=loaded_y_doc2vecS_pred_500_3000))

                                                                                  precision    recall  f1-score   support

                                              Інші-технічні-недоліки-стану-ліфту       0.33      0.56      0.41       180
                                  Будівництво-дооблаштування-дитячого-майданчику       0.59      0.84      0.69       180
Відновлення-благоустрою-після-вик--планових-аварійних-робіт-на-об-єктах-благоуст       0.55      0.68      0.61       369
                                                                    Відсутнє-ХВП       0.53      0.58      0.56       214
                                                            Відсутність-опалення       0.56      0.85      0.67       831
                                                  Відсутність-опалення-по-стояку       0.45      0.42      0.43       272
          Відсутність-освітлення-у-під-їзді-за-відсутності-несправності-лампочок       0.48      0.56      0.52       369
                       

In [26]:
#KNN_S_cosine_500_3000_10 = KNeighborsClassifier(n_neighbors=10, metric='cosine')
#KNN_S_cosine_500_3000_10.fit(X_doc2vecS_train_500_3000, y_train_500_3000) 
#pickle.dump(KNN_S_cosine_500_3000_10, open('netKNN_S_cosine_500_3000_10.pkl', 'wb'))
loaded_KNN_S_cosine_500_3000_10 = pickle.load(open('netKNN_S_cosine_500_3000_10.pkl', 'rb'))
loaded_KNN_S_cosine_500_3000_10

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [27]:
#y_doc2vecS_cosine_pred_500_3000_10 = KNN_S_cosine_500_3000_10.predict(X_doc2vecS_test_500_3000)
#pickle.dump(y_doc2vecS_cosine_pred_500_3000_10, open('y_doc2vecS_cosine_pred_500_3000_10.pkl', 'wb'))
loaded_y_doc2vecS_cosine_pred_500_3000_10 = pickle.load(open('y_doc2vecS_cosine_pred_500_3000_10.pkl', 'rb'))

In [28]:
#print(classification_report(y_true=y_test_500_3000, y_pred=y_doc2vecS_cosine_pred_500_3000_10))
print(classification_report(y_true=y_test_500_3000, y_pred=loaded_y_doc2vecS_cosine_pred_500_3000_10))

                                                                                  precision    recall  f1-score   support

                                              Інші-технічні-недоліки-стану-ліфту       0.40      0.78      0.53       180
                                  Будівництво-дооблаштування-дитячого-майданчику       0.71      0.87      0.78       180
Відновлення-благоустрою-після-вик--планових-аварійних-робіт-на-об-єктах-благоуст       0.59      0.78      0.67       369
                                                                    Відсутнє-ХВП       0.80      0.47      0.59       214
                                                            Відсутність-опалення       0.70      0.80      0.74       831
                                                  Відсутність-опалення-по-стояку       0.52      0.47      0.49       272
          Відсутність-освітлення-у-під-їзді-за-відсутності-несправності-лампочок       0.60      0.61      0.61       369
                       

In [29]:
print("Accuracy of the baseline model: {:.2f} %".format(sum(loaded_y_doc2vecS_cosine_pred_500_3000_10 == y_test_500_3000) / len(y_test_500_3000) * 100))

Accuracy of the baseline model: 67.05 %


### 2) Enhancement: 
    - Logistic Regression
    - TF-IDF
    - Random Forest
    - SVM


#### Logistic Regression model

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
lr_model = LogisticRegression()
lr_model.fit(X_doc2vecS_train_500_3000, y_train_500_3000)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [32]:
y_doc2vecS_pred_lr_500_3000 = lr_model.predict(X_doc2vecS_test_500_3000)

In [33]:
print(classification_report(y_true=y_test_500_3000, y_pred=y_doc2vecS_pred_lr_500_3000))

                                                                                  precision    recall  f1-score   support

                                              Інші-технічні-недоліки-стану-ліфту       0.64      0.61      0.63       180
                                  Будівництво-дооблаштування-дитячого-майданчику       0.87      0.83      0.85       180
Відновлення-благоустрою-після-вик--планових-аварійних-робіт-на-об-єктах-благоуст       0.78      0.74      0.76       369
                                                                    Відсутнє-ХВП       0.80      0.78      0.79       214
                                                            Відсутність-опалення       0.73      0.87      0.79       831
                                                  Відсутність-опалення-по-стояку       0.67      0.47      0.56       272
          Відсутність-освітлення-у-під-їзді-за-відсутності-несправності-лампочок       0.60      0.71      0.65       369
                       

In [34]:
print("Accuracy of the Logistic Regression model: {:.2f} %".format(sum(y_doc2vecS_pred_lr_500_3000 == y_test_500_3000) / len( y_test_500_3000) * 100))

Accuracy of the Logistic Regression model: 72.41 %


#### TF-IDF feature extraction

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [36]:
# Logistic Regression model on TF-IDF features
# Logistic Regression model on TF-IDF features
tv = TfidfVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1,2),sublinear_tf=True)
X_text_500_3000 = unpickled_df_500_3000['text'].tolist()
y_500_3000 = unpickled_df_500_3000['category'].tolist()

X_text_train_500_3000, X_text_test_500_3000, y_train_500_3000, y_test_500_3000 = train_test_split(
    X_text_500_3000, y_500_3000, test_size=0.33, random_state=42)
X_train_tfidf_500_3000_features = tv.fit_transform(X_text_train_500_3000)
X_test_tfidf_500_3000_features = tv.transform(X_text_test_500_3000)

In [37]:
lr = LogisticRegression(penalty='l2', max_iter=100, C=1)
lr.fit(X_train_tfidf_500_3000_features,y_train_500_3000)
y_lr_tfidf_pred_500_3000 = lr.predict(X_test_tfidf_500_3000_features)

In [38]:
print(classification_report(y_true=y_test_500_3000, y_pred=y_lr_tfidf_pred_500_3000))

                                                                                  precision    recall  f1-score   support

                                              Інші-технічні-недоліки-стану-ліфту       0.82      0.46      0.59       180
                                  Будівництво-дооблаштування-дитячого-майданчику       0.91      0.86      0.89       180
Відновлення-благоустрою-після-вик--планових-аварійних-робіт-на-об-єктах-благоуст       0.82      0.82      0.82       369
                                                                    Відсутнє-ХВП       0.97      0.84      0.90       214
                                                            Відсутність-опалення       0.68      0.96      0.80       831
                                                  Відсутність-опалення-по-стояку       0.76      0.32      0.45       272
          Відсутність-освітлення-у-під-їзді-за-відсутності-несправності-лампочок       0.64      0.89      0.75       369
                       

In [39]:
print("Logistic Regression model (TF-IDF features) accuracy: {:.2f} %".format(
    sum(y_lr_tfidf_pred_500_3000 == y_test_500_3000) / len(y_test_500_3000) * 100))

Logistic Regression model (TF-IDF features) accuracy: 76.57 %


#### Random Forest Classifier

In [40]:
from sklearn.ensemble import RandomForestClassifier

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [41]:
rf_model = RandomForestClassifier(n_estimators=100, max_depth=100)
rf_model.fit(X_train_tfidf_500_3000_features, y_train_500_3000)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [42]:
y_rfc_tfidf_pred_500_3000 = rf_model.predict(X_test_tfidf_500_3000_features)

In [43]:
print("Random Forest model (TF-IDF features) accuracy: {:.2f} %".format(
    sum(y_rfc_tfidf_pred_500_3000 == y_test_500_3000) / len(y_test_500_3000) * 100))

Random Forest model (TF-IDF features) accuracy: 69.93 %


In [44]:
rfc_model_vecS = RandomForestClassifier(n_estimators=100, max_depth=100)
rfc_model_vecS.fit(X_doc2vecS_train_500_3000, y_train_500_3000)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [45]:
y_rfc_doc2vecS_pred_500_3000 = rfc_model_vecS.predict(X_doc2vecS_test_500_3000)

In [46]:
print("Random Forest model (vector features) accuracy: {:.2f} %".format(
    sum(y_rfc_doc2vecS_pred_500_3000 == y_test_500_3000) / len(y_test_500_3000) * 100))

Random Forest model (vector features) accuracy: 60.01 %


#### SVM Multi-class classification with linear kernel (TF-IDF features)

In [47]:
from sklearn import svm

In [48]:
svm_model = svm.LinearSVC()
#svm_model.fit(X_train_tfidf_500_3000_features, y_train_500_3000)

In [49]:
#y_svm_tfidf_pred_500_3000 = svm_model.predict(X_test_tfidf_500_3000_features)

In [50]:
#pickle.dump(y_svm_tfidf_pred_500_3000, open('y_svm_tfidf_pred_500_3000.pkl', 'wb'))
#pickle.dump(svm_model, open('svm_model_tfidf_500_3000.pkl', 'wb'))

In [51]:
loaded_svm_model_tfidf_500_3000 = pickle.load(open('svm_model_tfidf_500_3000.pkl', 'rb'))
loaded_y_svm_tfidf_pred_500_3000= pickle.load(open('y_svm_tfidf_pred_500_3000.pkl', 'rb'))

In [52]:
print(classification_report(y_true=y_test_500_3000, y_pred=loaded_y_svm_tfidf_pred_500_3000))

                                                                                  precision    recall  f1-score   support

                                              Інші-технічні-недоліки-стану-ліфту       0.76      0.68      0.72       180
                                  Будівництво-дооблаштування-дитячого-майданчику       0.90      0.96      0.92       180
Відновлення-благоустрою-після-вик--планових-аварійних-робіт-на-об-єктах-благоуст       0.86      0.84      0.85       369
                                                                    Відсутнє-ХВП       0.95      0.95      0.95       214
                                                            Відсутність-опалення       0.80      0.93      0.86       831
                                                  Відсутність-опалення-по-стояку       0.73      0.50      0.59       272
          Відсутність-освітлення-у-під-їзді-за-відсутності-несправності-лампочок       0.67      0.82      0.74       369
                       

In [53]:
print("SVM model (TF-IDF features) accuracy: {:.2f} %".format(
    sum(loaded_y_svm_tfidf_pred_500_3000 == y_test_500_3000) / len(y_test_500_3000) * 100))

SVM model (TF-IDF features) accuracy: 82.31 %


In [54]:
svm_model_S = svm.LinearSVC()
svm_model_S.fit(X_doc2vecS_train_500_3000, y_train_500_3000)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [55]:
y_svm_doc2vecS_pred_500_3000 = svm_model_S.predict(X_doc2vecS_test_500_3000)

In [ ]:
pickle.dump(y_svm_doc2vecS_pred_500_3000, open('y_svm_doc2vecS_pred_500_3000.pkl', 'wb'))
pickle.dump(svm_model_S, open('svm_model_doc2vecS_500_3000.pkl', 'wb')) 
loaded_svm_model_doc2vecS_500_3000 = pickle.load(open('svm_model_doc2vecS_500_3000.pkl', 'rb'))
loaded_y_svm_doc2vecS_pred_500_3000= pickle.load(open('y_svm_doc2vecS_pred_500_3000.pkl', 'rb'))

In [155]:
print("SVM model (doc2vecS features) accuracy: {:.2f} %".format(
    sum(y_svm_doc2vecS_pred_500_3000 == y_test_500_3000) / len(y_test_500_3000) * 100))

SVM model (doc2vecS features) accuracy: 71.39 %


### 3). Using Neural Networks

#### Feedforward Neural Network


Also tried MLPClassifier for TFIdf extracted features but it throwed MemoryErrror. I think it would have better results then for doc2vec features, same as any other model.

In [57]:
from sklearn.neural_network import MLPClassifier

In [58]:
#nn = MLPClassifier(hidden_layer_sizes=(1000, 1000,), activation='relu', solver='adam', batch_size = 64)

In [59]:
#nn.fit(X_doc2vecS_train_500_3000, y_train_500_3000)
#y_doc2vecS_pred_nn = nn.predict(X_doc2vecS_test_500_3000)
#pickle.dump(y_doc2vecS_pred_nn, open('y_doc2vecS_pred_nn_500_3000.pkl', 'wb'))
#pickle.dump(nn, open('nn_doc2vecS_500_3000.pkl', 'wb')) 

In [60]:
loaded_nn_doc2vecS_500_3000 = pickle.load(open('nn_doc2vecS_500_3000.pkl', 'rb'))
loaded_y_doc2vecS_pred_nn_500_3000= pickle.load(open('y_doc2vecS_pred_nn_500_3000.pkl', 'rb'))

In [61]:
print(classification_report(y_true=y_test_500_3000, y_pred=loaded_y_doc2vecS_pred_nn_500_3000))

                                                                                  precision    recall  f1-score   support

                                              Інші-технічні-недоліки-стану-ліфту       0.65      0.58      0.61       180
                                  Будівництво-дооблаштування-дитячого-майданчику       0.95      0.72      0.82       180
Відновлення-благоустрою-після-вик--планових-аварійних-робіт-на-об-єктах-благоуст       0.70      0.80      0.74       369
                                                                    Відсутнє-ХВП       0.83      0.86      0.85       214
                                                            Відсутність-опалення       0.83      0.80      0.82       831
                                                  Відсутність-опалення-по-стояку       0.55      0.61      0.58       272
          Відсутність-освітлення-у-під-їзді-за-відсутності-несправності-лампочок       0.61      0.75      0.67       369
                       

In [156]:
print("Feedforward NN model (doc2vecS features) accuracy: {:.2f} %".format(
    sum(loaded_y_doc2vecS_pred_nn_500_3000 == y_test_500_3000) / len(y_test_500_3000) * 100))

Feedforward NN model (doc2vecS features) accuracy: 74.06 %


#### LSTM

In [164]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping

from keras.utils.np_utils import to_categorical

In [165]:
TOP_N_WORDS = 20000
SEQUENCE_LENGTH = 300
EMBEDDING_DIMENTION = 100

In [166]:
def tokenizetext(text, vectors):
    text=re.sub(r'[.\-\d]', " ", text)
    tokens = [morph.parse(i.lower().strip(r"\d"))[0].normal_form for i in word_tokenize(text)
                  if morph.parse(i.lower()) and not i.lower().strip("-") in stop_words]
    return tokens

In [167]:
#unpickled_df_500_3000_tokenized=pd.read_pickle("./embedded_500_3000.pkl")
#unpickled_df_500_3000_tokenized.loc[:, 'tokenized_text'] = unpickled_df_500_3000_tokenized.apply(lambda x: tokenizetext(x['text'], uk_vectors), axis=1)
#unpickled_df_500_3000_tokenized.to_pickle("./embedded_500_3000_tokenized.pkl")
unpickled_df_500_3000_tokenized=pd.read_pickle("./embedded_500_3000_tokenized.pkl")

In [168]:
unpickled_df_500_3000_tokenized

,category,text,id,doc2vec,doc2w2vec,doc2vec_reshape,doc2w2vec_reshape,doc2vecS,doc2w2vecS,doc2vecS_reshape,doc2w2vecS_reshape,tokenized_text
4816,Прибирання-приміщень,В підїзді №1 не прибирається..,3067803,"[[-0.084547, -0.001392, -0.058494, 0.207588, -0.010897, -0.044266, -0.047631, 0.06342, 0.005104, 0.089637, 0.070812, 0.008774, -0.015949, 0.012546, 0.190972, 0.054239, 0.22103, -0.00713, -0.142183, 0.080955, -0.031203, 0.109135, 0.165411, -0.045446, -0.181592, 0.045766, -0.097201, -0.010174, -0....","[[-2.336758, -0.501181, 0.457877, -1.358576, 0.041353, -2.046993, 0.023815, 1.547083, -1.742819, -2.262023, -1.506638, -6.017622, 0.420171, -2.292722, 2.393311, 0.269985, 1.113025, -0.598293, -1.889485, -0.106864, -1.201012, -1.16625, 2.016952, 0.122736, -0.926234, -0.598504, -1.026642, -4.13740...","[-0.084547, -0.001392, -0.058494, 0.207588, -0.010897, -0.044266, -0.047631, 0.06342, 0.005104, 0.089637, 0.070812, 0.008774, -0.015949, 0.012546, 0.190972, 0.054239, 0.22103, -0.00713, -0.142183, 0.080955, -0.031203, 0.109135, 0.165411, -0.045446, -0.181592, 0.045766, -0.097201, -0.010174, -0.0...","[-2.336758, -0.501181, 0.457877, -1.358576, 0.041353, -2.046993, 0.023815, 1.547083, -1.742819, -2.262023, -1.506638, -6.017622, 0.420171, -2.292722, 2.393311, 0.269985, 1.113025, -0.598293, -1.889485, -0.106864, -1.201012, -1.16625, 2.016952, 0.122736, -0.926234, -0.598504, -1.026642, -4.137404...","[[-0.084547, -0.001392, -0.058494, 0.207588, -0.010897, -0.044266, -0.047631, 0.06342, 0.005104, 0.089637, 0.070812, 0.008774, -0.015949, 0.012546, 0.190972, 0.054239, 0.22103, -0.00713, -0.142183, 0.080955, -0.031203, 0.109135, 0.165411, -0.045446, -0.181592, 0.045766, -0.097201, -0.010174, -0....","[[-2.336758, -0.501181, 0.457877, -1.358576, 0.041353, -2.046993, 0.023815, 1.547083, -1.742819, -2.262023, -1.506638, -6.017622, 0.420171, -2.292722, 2.393311, 0.269985, 1.113025, -0.598293, -1.889485, -0.106864, -1.201012, -1.16625, 2.016952, 0.122736, -0.926234, -0.598504, -1.026642, -4.13740...","[-0.084547, -0.001392, -0.058494, 0.207588, -0.010897, -0.044266, -0.047631, 0.06342, 0.005104, 0.089637, 0.070812, 0.008774, -0.015949, 0.012546, 0.190972, 0.054239, 0.22103, -0.00713, -0.142183, 0.080955, -0.031203, 0.109135, 0.165411, -0.045446, -0.181592, 0.045766, -0.097201, -0.010174, -0.0...","[-2.336758, -0.501181, 0.457877, -1.358576, 0.041353, -2.046993, 0.023815, 1.547083, -1.742819, -2.262023, -1.506638, -6.017622, 0.420171, -2.292722, 2.393311, 0.269985, 1.113025, -0.598293, -1.889485, -0.106864, -1.201012, -1.16625, 2.016952, 0.122736, -0.926234, -0.598504, -1.026642, -4.137404...","[підїзда, №, не, прибиратися]"
4817,Прибирання-приміщень,"1. Прибирання підїзду робиться вкрай нерегулярно. Іноді протягом 3-х тижнів, або навіть місяця. 2. Косметичний ремонт підїзду на останньому поверсі відсутній.",2937422,"[[0.500563, 0.286424, 0.532075, 0.46481, 0.229421, 0.553665, 0.150944, 0.159035, 0.506101, 0.13541, 0.661964, 0.157294, 0.460409, 0.570167, 1.081728, 0.52474, 0.437801, 0.293852, 0.283891, 0.387058, 0.401101, 0.262813, 0.141715, 0.118816, 0.205297, 0.353372, 0.49583, 0.275393, 0.295218, 0.344787...","[[0.341461, 1.204187, 1.925844, 2.655365, 2.204645, 2.012463, 1.843585, 3.687111, 1.211816, 1.546087, 1.323149, 1.915068, 2.61784, 1.563169, 1.592014, 2.853098, 1.864407, 1.802295, 1.56014, 2.49524, 3.138227, 0.259099, 1.280812, 1.664527, 1.057407, 2.424162, 2.103551, 5.108227, 3.080941, 2.61125...","[0.500563, 0.286424, 0.532075, 0.46481, 0.229421, 0.553665, 0.150944, 0.159035, 0.506101, 0.13541, 0.661964, 0.157294, 0.460409, 0.570167, 1.081728, 0.52474, 0.437801, 0.293852, 0.283891, 0.387058, 0.401101, 0.262813, 0.141715, 0.118816, 0.205297, 0.353372, 0.49583, 0.275393, 0.295218, 0.344787,...","[0.341461, 1.204187, 1.925844, 2.655365, 2.204645, 2.012463, 1.843585, 3.687111, 1.211816, 1.546087, 1.323149, 1.915068, 2.61784, 1.563169, 1.592014, 2.853098, 1.864407, 1.802295, 1.56014, 2.49524, 3.138227, 0.259099, 1.280812, 1.664527, 1.0574

In [169]:
tokenizer = Tokenizer(num_words=TOP_N_WORDS)
tokenizer.fit_on_texts(unpickled_df_500_3000['text'].values)

In [170]:
word_index = tokenizer.word_index
len(word_index)

82943

In [171]:
X_tokenized = tokenizer.texts_to_sequences(unpickled_df_500_3000['text'].values)
X_tokenized = pad_sequences(X_tokenized, maxlen=SEQUENCE_LENGTH)
print('Tokenized data shape:', X_tokenized.shape)

Tokenized data shape: (20395, 300)


In [172]:
y_dummies_500_3000 = pd.get_dummies(unpickled_df_500_3000['category']).values
print('Dummies vector shape:', y_dummies_500_3000.shape)

Dummies vector shape: (20395, 22)


In [173]:
#X_tokenized_train_500_3000 = X_tokenized[:int(13664)]
#X_tokenized_test_500_3000  = X_tokenized[int(13664):]

#y_dummies_train_500_3000  = y_dummies_500_3000[:int(13664)]
#y_dummies_test_500_3000  = y_dummies_500_3000[int(13664):]

In [174]:
X_tokenized_train_500_3000, X_tokenized_test_500_3000, y_dummies_train_500_3000, y_dummies_test_500_3000 = train_test_split(
    X_tokenized, y_dummies_500_3000, test_size=0.33, random_state=42)

In [175]:
print("Train shape"+str(X_tokenized_train_500_3000.shape))
print("Test shape"+str(X_tokenized_test_500_3000.shape))

Train shape(13664, 300)
Test shape(6731, 300)


In [176]:
LSTM_tokenized_model = Sequential()
LSTM_tokenized_model.add(Embedding(TOP_N_WORDS, EMBEDDING_DIMENTION, input_length=X_tokenized_train_500_3000.shape[1]))
LSTM_tokenized_model.add(SpatialDropout1D(0.2))
LSTM_tokenized_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
LSTM_tokenized_model.add(Dense(22, activation='softmax'))
LSTM_tokenized_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(LSTM_tokenized_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 300, 100)          2000000   
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 300, 100)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_5 (Dense)              (None, 22)                2222      
Total params: 2,082,622
Trainable params: 2,082,622
Non-trainable params: 0
_________________________________________________________________
None


In [223]:
#history = LSTM_tokenized_model.fit(X_tokenized_train_500_3000, 
#                    y_dummies_train_500_3000, 
#                    epochs=20, 
#                    batch_size=60,
#                    validation_split=0.1,
#                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

*Picture of saved history of pickled LSTM* :

<img src="lstm_tr.png">

In [178]:
#y_tokenized_lstm_pred_500_3000 = LSTM_tokenized_model.predict(X_tokenized_test_500_3000)

In [180]:
#pickle.dump(y_tokenized_lstm_pred_500_3000, open('y_tokenized_lstm_pred_500_3000_2.pkl', 'wb'))
#pickle.dump(LSTM_tokenized_model, open('LSTM_tokenized_model_500_3000_2.pkl', 'wb')) 
loaded_LSTM_tokenized_model_500_3000 = pickle.load(open('LSTM_tokenized_model_500_3000_2.pkl', 'rb'))
loaded_y_tokenized_lstm_pred_500_3000= pickle.load(open('y_tokenized_lstm_pred_500_3000_2.pkl', 'rb'))

In [179]:
y_tokenized_lstm_pred_500_3000 = y_tokenized_lstm_pred_500_3000.argmax(axis=1)
y_dummies_test_500_3000_single = y_dummies_test_500_3000.argmax(axis=1)

In [183]:
print(classification_report(y_true=y_dummies_test_500_3000_single , y_pred=loaded_y_tokenized_lstm_pred_500_3000))

             precision    recall  f1-score   support

          0       0.47      0.57      0.51       180
          1       0.82      0.78      0.80       180
          2       0.67      0.70      0.68       369
          3       0.80      0.73      0.76       214
          4       0.79      0.86      0.82       831
          5       0.54      0.39      0.45       272
          6       0.57      0.75      0.65       369
          7       0.65      0.76      0.70       231
          8       0.72      0.66      0.69       326
          9       0.77      0.84      0.81       297
         10       0.57      0.43      0.49       167
         11       0.39      0.24      0.30       218
         12       0.86      0.89      0.87       646
         13       0.62      0.57      0.60       181
         14       0.66      0.50      0.57       277
         15       0.56      0.58      0.57       194
         16       0.75      0.78      0.77       491
         17       0.35      0.38      0.37   

In [184]:
print("LSTM model (tokenized features) accuracy: {:.2f} %".format(
    sum(loaded_y_tokenized_lstm_pred_500_3000 == y_dummies_test_500_3000_single ) / len(y_dummies_test_500_3000 ) * 100))

LSTM model (tokenized features) accuracy: 68.89 %


### Ensemble of best models

In [221]:
best_model_pred_500_3000 = pd.DataFrame({
    'KNN': loaded_y_doc2vecS_cosine_pred_500_3000_10,
    'LR_TFIDF':y_lr_tfidf_pred_500_3000,
    'RF_TFIDF': y_rfc_tfidf_pred_500_3000 , 
    'SVM_TFIDF': loaded_y_svm_tfidf_pred_500_3000, 
    'NN_Doc2VecS': loaded_y_doc2vecS_pred_nn_500_3000
    #'LSTM_tokenized':loaded_y_tokenized_lstm_pred_500_3000
})

best_model_pred_500_3000['y']=y_test_500_3000
#best_model_pred_500_3000['y_dummies']=y_dummies_test_500_3000_single
best_model_pred_500_3000['y_pred_ensemble']=[''] * len(y_test_500_3000)


for j, name in tqdm(zip([0,1,2,3,4,5], ['KNN', 'LR_TFIDF', 'RF_TFIDF','SVM_TFIDF', 'SVM_TFIDF','NN_Doc2VecS'])):
    for i in range(len(best_model_pred_500_3000)):
        values = best_model_pred_500_3000.loc[i, ['KNN', 'LR_TFIDF', 'RF_TFIDF','SVM_TFIDF','SVM_TFIDF','NN_Doc2VecS']].values
        #print(values)
        #print(Counter(values))
        #print(np.bincount(values))
        best_model_pred_500_3000.loc[i, 'y_pred_ensemble'] = Counter(values).most_common(1)[0][0]





0it [00:00, ?it/s]



1it [00:25, 25.18s/it]



2it [00:52, 25.72s/it]



3it [01:17, 25.60s/it]



4it [01:44, 25.92s/it]



5it [02:10, 26.11s/it]



6it [02:36, 25.88s/it]

In [222]:
print("Ensemble model accuracy ({}): {:.2f} %".format(name, sum(best_model_pred_500_3000['y_pred_ensemble'] == y_test_500_3000) / len(y_test_500_3000) * 100))

Ensemble model accuracy (NN_Doc2VecS): 80.85 %
